<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW13/NLP_DF_HW13_paraphrase1eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset

my_dataset_dictionary = load_dataset("humarin/chatgpt-paraphrases")
my_dataset_dictionary['train'][0]

Found cached dataset csv (/Users/evamelissatasdemir/.cache/huggingface/datasets/humarin___csv/humarin--chatgpt-paraphrases-56e87c47b965542a/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

{'text': 'What is the step by step guide to invest in share market in india?',
 'paraphrases': "['Can you provide a detailed procedure for investing in the Indian stock market?', 'What are the sequential instructions for investing in shares in India?', 'Could you outline the step-by-step process for investing in the Indian share market?', 'What is the systematic guide to investing in the Indian stock exchange?', 'Can you provide a comprehensive guide on how to invest in the Indian share market?']",
 'category': 'question',
 'source': 'quora'}

In [ ]:
my_dataset_dictionary

DatasetDict({
    train: Dataset({
        features: ['text', 'paraphrases', 'category', 'source'],
        num_rows: 419197
    })
})

In [ ]:
my_dataset_dictionary['train']

Dataset({
    features: ['text', 'paraphrases', 'category', 'source'],
    num_rows: 419197
})

In [ ]:
my_dataset_dictionary['train']['paraphrases'][1]

"['Can you tell me about the history of the Kohinoor (Koh-i-Noor) Diamond?', 'What is the tale behind the Kohinoor (Koh-i-Noor) Diamond?', 'Could you narrate the story of the Kohinoor (Koh-i-Noor) Diamond?', 'What is the account of the Kohinoor (Koh-i-Noor) Diamond?', 'Can you describe the legend of the Kohinoor (Koh-i-Noor) Diamond?']"

In [ ]:
my_dataset_dictionary['train']['text'][1]

'What is the story of Kohinoor (Koh-i-Noor) Diamond?'

In [ ]:
import torch

@property
def device(self) -> torch.device:
        if torch.cuda.is_available():
            return torch.device("cuda")
        elif torch.backends.mps.is_available():
            return torch.device("mps")
        else:

            return torch.device("cpu")

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'humarin/chatgpt_paraphraser_on_T5_base'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.to('mps');
model.eval();


def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(paraphrase('Germany will pay Holocaust survivors over $1.4 billion next year.'))

Germany will provide more than $1.4 billion in financial support to Holocaust survivors in the next year, as the country prepares to pay more than


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device='mps'

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
# batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to('mps')

def paraphrase(

    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128

):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to('mps')

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty,
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [ ]:
text = 'What are the best places to see in New York?'
paraphrase(text)


['What are some must-see places in New York?',
 'Can you suggest some must-see spots in New York?',
 'Where should one go to experience the best NYC has to offer?',
 'Which places should I visit in New York?',
 'What are the top destinations to explore in New York?']

In [ ]:
text = "Rammstein's album Mutter was recorded in the south of France in May and June 2000, and mixed in Stockholm in October of that year."
paraphrase(text)


['In May and June 2000, Rammstein travelled to the south of France to record his album Mutter, which was mixed in Stockholm in October of that year.',
 'The album Mutter by Rammstein was recorded in the south of France during May and June 2000, with mixing taking place in Stockholm in October of that year.',
 'The album Mutter by Rammstein was recorded in the south of France during May and June 2000, with mixing taking place in Stockholm in October of that year. It',
 'Mutter, the album released by Rammstein, was recorded in southern France during May and June 2000, with mixing taking place between October and September.',
 'In May and June 2000, Rammstein recorded his album Mutter in the south of France, with the mix being made at Stockholm during October.']